# 

In [ ]:
# unpack data
# !tar -xzvf JEOPARDY_QUESTIONS1.json.tar.gz

In [2]:
reset -fs

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

## Load and Prep Data

In [5]:
df = pd.read_json("JEOPARDY_QUESTIONS1.json")

In [6]:
df.head(10)

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
5,3-LETTER WORDS,2004-12-31,"'In the title of an Aesop fable, this insect s...",$200,the ant,Jeopardy!,4680
6,HISTORY,2004-12-31,'Built in 312 B.C. to link Rome & the South of...,$400,the Appian Way,Jeopardy!,4680
7,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 8: 30 steals for the Birmingham Barons; 2...,$400,Michael Jordan,Jeopardy!,4680
8,EVERYBODY TALKS ABOUT IT...,2004-12-31,"'In the winter of 1971-72, a record 1,122 inch...",$400,Washington,Jeopardy!,4680
9,THE COMPANY LINE,2004-12-31,'This housewares store was named for the packa...,$400,Crate & Barrel,Jeopardy!,4680


In [55]:
df['value'].unique()

array(['$200', '$400', '$600', '$800', '$2,000', '$1000', '$1200',
       '$1600', '$2000', '$3,200', None, '$5,000', '$100', '$300', '$500',
       '$1,000', '$1,500', '$1,200', '$4,800', '$1,800', '$1,100',
       '$2,200', '$3,400', '$3,000', '$4,000', '$1,600', '$6,800',
       '$1,900', '$3,100', '$700', '$1,400', '$2,800', '$8,000', '$6,000',
       '$2,400', '$12,000', '$3,800', '$2,500', '$6,200', '$10,000',
       '$7,000', '$1,492', '$7,400', '$1,300', '$7,200', '$2,600',
       '$3,300', '$5,400', '$4,500', '$2,100', '$900', '$3,600', '$2,127',
       '$367', '$4,400', '$3,500', '$2,900', '$3,900', '$4,100', '$4,600',
       '$10,800', '$2,300', '$5,600', '$1,111', '$8,200', '$5,800',
       '$750', '$7,500', '$1,700', '$9,000', '$6,100', '$1,020', '$4,700',
       '$2,021', '$5,200', '$3,389', '$4,200', '$5', '$2,001', '$1,263',
       '$4,637', '$3,201', '$6,600', '$3,700', '$2,990', '$5,500',
       '$14,000', '$2,700', '$6,400', '$350', '$8,600', '$6,300', '$250',
      

In [56]:
df['air_date']= pd.to_datetime(df['air_date'])

In [59]:
df.head(30)

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
5,3-LETTER WORDS,2004-12-31,"'In the title of an Aesop fable, this insect s...",$200,the ant,Jeopardy!,4680
6,HISTORY,2004-12-31,'Built in 312 B.C. to link Rome & the South of...,$400,the Appian Way,Jeopardy!,4680
7,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 8: 30 steals for the Birmingham Barons; 2...,$400,Michael Jordan,Jeopardy!,4680
8,EVERYBODY TALKS ABOUT IT...,2004-12-31,"'In the winter of 1971-72, a record 1,122 inch...",$400,Washington,Jeopardy!,4680
9,THE COMPANY LINE,2004-12-31,'This housewares store was named for the packa...,$400,Crate & Barrel,Jeopardy!,4680


In [ ]:

# first round values, earliest show in data
# could also do show_number==1
df['air_date'].min(), df[(df['air_date']==df['air_date'].min()) & (df['round']=='Jeopardy!')]['value'].value_counts()

(Timestamp('1984-09-10 00:00:00'),
 $100    5
 $200    5
 $400    5
 $300    4
 $500    3
 $800    1
 Name: value, dtype: int64)

In [ ]:
# example final jeopardy row, last show in data
df[(df['show_number']==4999) & (df['round']=='Final Jeopardy!')]

,category,air_date,question,value,answer,round,show_number
216929,HISTORIC NAMES,2006-05-11,'A silent movie title includes the last name o...,None,Grigori Alexandrovich Potemkin,Final Jeopardy!,4999


In [ ]:
# Existing category counts, desc
df.groupby('category').count()[['round']].sort_values(by=['round'], ascending=False).head(10)

,round
category,
BEFORE & AFTER,547
SCIENCE,519
LITERATURE,496
AMERICAN HISTORY,418
POTPOURRI,401
WORLD HISTORY,377
WORD ORIGINS,371
COLLEGES & UNIVERSITIES,351
HISTORY,349


Some cleanup/info
- [x] air-date to datetime
- [x] encode round
    - Daily Doubles are implicit from money value - Ignore
    - 3 Tiebreakers - Drop
- [ ] value to int/standardize money,  historical info [link](https://outsider.com/news/entertainment/jeopardy-show-double-dollar-values-categories/)
    - final question's have value np.nan
    - Rank question difficulty 1 through 11 (5 qn values 1st round, 5 qn values 2nd round, 1 final jeopardy)?
    - Or 1 through 13: 5 normal qn types plus daily double per round, times 2, and 1 final jeopardy
    - ~~for the pilot, questions values were 25-125 for first round, then doubled~~ pilot not in the data
    - Jeopardy! round was 100-500 for years
    - 11/26/01 doubled all monetary values, so 200-1000 for Jeopardy! round

In [ ]:
df = df[df['round']!='Tiebreaker']

In [ ]:
round_dict = {}
for idx, jp_round in enumerate(df['round'].unique()):
    round_dict[jp_round] = idx
round_dict

{'Jeopardy!': 0, 'Double Jeopardy!': 1, 'Final Jeopardy!': 2}

In [ ]:
# df['enc_round'] = df['round'].map(round_dict)
df.replace({'round': round_dict}, inplace=True)

In [ ]:
df.head(2)

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,0,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,0,4680


In [ ]:
f"There are {df['category'].nunique():,} unique categories."

'There are 27,995 unique categories.'

In [ ]:
cts = df.groupby('category').count()
print(f"There are {cts[cts < 100].count()[0]:,} categories with less than 100 questions.")
print(f"There are {cts[cts >= 100].count()[0]:} categories with more than 100 questions.")

There are 27,845 categories with less than 100 questions.
There are 150 categories with more than 100 questions.


### Data Preprocessing
Here we need to use TFIDF to get the term-document matrix, then we can directly feed that into an NMF or LDA model.

In [18]:
qs = df['question']
samp = qs.sample(frac=0.1)

In [21]:
print(samp.shape[0])

21693


In [22]:
vectorizer = TfidfVectorizer(strip_accents='ascii', stop_words='english', max_df=0.95)  # we may need to find better stop words
tfidf_qs = vectorizer.fit_transform(samp)

In [25]:
tfidf_qs.shape  # 30,886 words in this sample

(21693, 30886)

Maybe we need to clean up the questions first so we don't get weird words like these. One way could be to increase 'max_df' for the vectorizer.

In [31]:
sorted(vectorizer.get_feature_names())[:25] 

['00',
 '000',
 '0005',
 '000th',
 '002',
 '007',
 '01',
 '01_dj_01',
 '01_dj_03',
 '01_dj_04',
 '01_dj_05',
 '01_dj_05a',
 '01_dj_06',
 '01_dj_07',
 '01_dj_08',
 '01_dj_10',
 '01_dj_13',
 '01_dj_14',
 '01_dj_14a',
 '01_dj_16',
 '01_dj_17',
 '01_dj_17a',
 '01_dj_20',
 '01_dj_21',
 '01_dj_23']

In [50]:
# look at the non-words in an individual question
np_tfidf = tfidf_qs.toarray()
q1 = np.argwhere(np_tfidf[0] > 0).flatten()

idx_to_word = {idx: word for word, idx in vectorizer.vocabulary_.items()}
list(map(idx_to_word.get, q1))

['batter',
 'beef',
 'cooked',
 'dipped',
 'includes',
 'meat',
 'pounded',
 'skillet']

In [54]:
samp

117603    'Beef pounded thin, dipped in batter & cooked ...
200136    '(<a href="http://www.j-archive.com/media/2009...
142718    'This Christian church is headed by The Patria...
108186    'If Ms. Lee, lovely co-hostess of "High Roller...
5428      'A woman claiming to be this Lewis & Clark com...
                                ...                        
109687                          'Pestle's partner<br />(6)'
28269     'Beginning a program to modernize the nation, ...
29957                  'Hamlet, or Hans Christian Andersen'
73618     'The fanged ones diligently recycle No. 1 (PET...
97167     'The Spanish had fortified San Juan Hill, so t...
Name: question, Length: 21693, dtype: object

In [51]:
model = NMF(n_components=25, init='random', verbose=100)
W = model.fit_transform(np_tfidf)  # document-topic matrix, rows=docs, cols=topic
H = model.components_  # topic-term matrix, rows=topic, cols=terms

KeyboardInterrupt: 

In [65]:
NMF().get_params()

{'alpha': 0.0,
 'beta_loss': 'frobenius',
 'init': 'warn',
 'l1_ratio': 0.0,
 'max_iter': 200,
 'n_components': None,
 'random_state': None,
 'regularization': 'both',
 'shuffle': False,
 'solver': 'cd',
 'tol': 0.0001,
 'verbose': 0}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eebde281-32de-46a2-88ad-b5936664574a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>